In [ ]:
from __future__ import print_function
#import Keras library
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.metrics import categorical_accuracy

#import spacy, and spacy french model
# spacy is used to work on text
import spacy
nlp = spacy.load('en')

#import other libraries
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle

Using TensorFlow backend.


#Reading data and creating wordlist

In [ ]:
def create_wordlist(doc):
    wl = []
    for word in doc:
        if word.text not in ("\n","\n\n",'\u2009','\xa0'):
            wl.append(word.text.lower())
    return wl

In [ ]:
wordlist = []

In [ ]:
with codecs.open('/content/elontweets_preprocessed.txt', "r") as f:
        data = f.read()
        #create sentences
        doc = nlp(data)
        wl = create_wordlist(doc)
        wordlist = wordlist + wl

In [ ]:
print(wordlist)

['assuming', 'max', 'acceleration', 'of', '2', 'to', '3', 'g', 's', 'but', 'in', 'a', 'comfortable', 'direction', 'will', 'feel', 'like', 'a', 'mild', 'to', 'moder', 'rt', 'bfr', 'is', 'capable', 'of', 'transporting', 'satellites', 'to', 'orbit', 'crew', 'and', 'cargo', 'to', 'the', 'station', 'and', 'completing', 'missions', 'to', 'the', 'moon', 'an', 'yup', 'part', '2', 'fly', 'to', 'most', 'places', 'on', 'earth', 'in', 'under', '30', 'mins', 'and', 'anywhere', 'in', 'under', '60', 'cost', 'per', 'seat', 'should', 'be', 'rt', 'supporting', 'the', 'creation', 'of', 'a', 'permanent', 'self', 'sustaining', 'human', 'presence', 'on', 'mars', 'bfr', 'will', 'take', 'you', 'anywhere', 'on', 'earth', 'in', 'less', 'than', '60', 'mins', 'mars', 'city', 'opposite', 'of', 'earth', 'dawn', 'and', 'dusk', 'sky', 'are', 'blue', 'on', 'mars', 'and', 'day', 'sky', 'is', 'red', 'moon', 'base', 'alpha', 'will', 'be', 'announcing', 'something', 'really', 'special', 'at', 'today', 's', 'talk', 'rt', '

#Creating Dictionary

In [ ]:
save_dir = '/content/'
vocab_file = os.path.join(save_dir, "words_vocab.pkl")

In [ ]:
ls

elontweets_preprocessed.txt  sample_data/  words_vocab.pkl


In [ ]:
# count the number of words
word_counts = collections.Counter(wordlist)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("vocab size: ", vocab_size)

#save the words and vocabulary
with open(os.path.join(vocab_file), 'wb') as f:
    cPickle.dump((words, vocab, vocabulary_inv), f)

vocab size:  6341


#create sentence list

In [ ]:
sequences_step = 1 #step to create sequences
seq_length = 30

In [ ]:
sequences = []
next_words = []
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])
    next_words.append(wordlist[i + seq_length])

print('nb sequences:', len(sequences))

nb sequences: 42269


In [ ]:
X = np.zeros((len(sequences), seq_length, vocab_size), dtype=np.bool)
y = np.zeros((len(sequences), vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1

#Building Bidirectional LSTM,finally!

In [ ]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    print("model built!")
    return model

In [ ]:
rnn_size = 256 # size of RNN
seq_length = 30 # sequence length
learning_rate = 0.001 #learning rate

md = bidirectional_lstm_model(seq_length, vocab_size)
md.summary()

Build LSTM model.
model built!
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               13512704  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6341)              3252933   
_________________________________________________________________
activation_1 (Activation)    (None, 6341)              0         
Total params: 16,765,637
Trainable params: 16,765,637
Non-trainable params: 0
_________________________________________________________________


#Training time!


In [ ]:
batch_size = 32 # minibatch size
num_epochs = 100 # number of epochs

callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath=save_dir + "/" + 'my_model_gen_sentences.{epoch:02d}-{val_loss:.2f}.hdf5',\
                           monitor='val_loss', verbose=0, mode='auto', period=2)]
#fit the model
history = md.fit(X, y,
                 batch_size=batch_size,
                 shuffle=True,
                 epochs=num_epochs,
                 callbacks=callbacks,
                 validation_split=0.1)

#save the model
md.save(save_dir + "/" + 'my_model_generate_sentences.h5')

Train on 38042 samples, validate on 4227 samples
Epoch 1/100
  320/38042 [..............................] - ETA: 38:41 - loss: 8.7484 - categorical_accuracy: 0.0188

In [ ]:
#load vocabulary
print("loading vocabulary...")
vocab_file = os.path.join(save_dir, "words_vocab.pkl")

with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        words, vocab, vocabulary_inv = cPickle.load(f)

vocab_size = len(words)

from keras.models import load_model
# load the model
print("loading model...")
model = load_model(save_dir + "/" + 'my_model_generate_sentences.h5')

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
words_number = 10 # number of words to generate
seed_sentences = "launch is going to be " #seed sentence to start the generating.

#initiate sentences
generated = ''
sentence = []

#we shate the seed accordingly to the neural netwrok needs:
for i in range (seq_length):
    sentence.append("a")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)

#the, we generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1.

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 0.7)
    next_word = vocabulary_inv[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

#print the whole text
print(generated)

In [ ]:
len(vocabulary_inv)